In [1]:
import numpy as np

import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt # side-stepping mpl backend

from nltk.stem.lancaster import LancasterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
import heapq

from nltk.corpus import stopwords

vectorizer = CountVectorizer()
st = LancasterStemmer()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open("TestData/train.dat", "r") as fh:
    #with open("TestData/Test/training_out.dat", "r") as fh:
    linesOfTrainData = fh.readlines()
print(len(linesOfTrainData))

#with open("TestData/format.dat", "r") as fh:
with open("TestData/Test/format_out.dat", "r") as fh:
    linesOfFormat = fh.readlines()
print(len(linesOfFormat))

with open("TestData/test.dat", "r") as fh:
#with open("TestData/Test/test_out.dat", "r") as fh:
    linesOfTestData = fh.readlines()
print(len(linesOfTestData))


18506
6
18506


In [3]:
##stopwords and stemming for Training

print ("First line before cleanup from Training Data: ",linesOfTrainData[0])

stops = set(stopwords.words('english'))

linesOfTrainDataAfterPreProcessing=[]
for line in linesOfTrainData:
    newLine = line
    for w in newLine.split():
        if w.lower() in stops:
            newLine = newLine.replace(' '+w+' ', ' ') # for identifting words
    linesOfTrainDataAfterPreProcessing.append(newLine)


linesOfTrainDataAfterSteming = []
for line in linesOfTrainDataAfterPreProcessing:
    newLine = line
    for w in newLine.split():
            newLine = newLine.replace(w, st.stem(w)) # for stemming
    linesOfTrainDataAfterSteming.append(newLine)

linesOfTrainData = linesOfTrainDataAfterSteming

print ("\n\nFirst line after cleanup from Training Data: ",linesOfTrainData[0])

First line before cleanup from Training Data:  +1	This book is such a life saver.  It has been so helpful to be able to go back to track trends, answer pediatrician questions, or communicate with each other when you are up at different times of the night with a newborn.  I think it is one of those things that everyone should be required to have before they leave the hospital.  We went through all the pages of the newborn version, then moved to the infant version, and will finish up the second infant book (third total) right as our baby turns 1.  See other things that are must haves for baby at [...]



First line after cleanup from Training Data:  +1	thi book lif saver.  help abl go back track trends, answ pedy questions, commun diff tim night newborn.  think on thing everyon requir leav hospital.  went pag newborn version, mov inf version, fin second inf book (third total) right baby turn 1.  see thing must hav baby [...]



In [4]:
##stopwords and stemming for Testdata

print ("First line before cleanup from Test Data: ",linesOfTestData[0])

stops = set(stopwords.words('english'))

linesOfTestDataAfterPreProcessing=[]
for line in linesOfTestData:
    newLine = line
    for w in newLine.split():
        if w.lower() in stops:
            newLine = newLine.replace(' '+w+' ', ' ') # for identifting words
    linesOfTestDataAfterPreProcessing.append(newLine)


linesOfTestDataAfterSteming = []
for line in linesOfTestDataAfterPreProcessing:
    newLine = line
    for w in newLine.split():
            newLine = newLine.replace(w, st.stem(w)) # for stemming
    linesOfTestDataAfterSteming.append(newLine)

linesOfTestData = linesOfTestDataAfterSteming

print ("\n\nFirst line after cleanup from Test Data: ",linesOfTestData[0])

First line before cleanup from Test Data:  Perfect for new parents. We were able to keep track of baby's feeding, sleep and diaper change schedule for the first two and a half months of her life. Made life easier when the doctor would ask questions about habits because we had it all right there!



First line after cleanup from Test Data:  perfect new parents. abl keep track baby's feeding, sleep diap chang schedule first two half month lif. mad lif easy doct would ask quest habit right there!



In [5]:
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(linesOfTrainData, linesOfTestData, test_size= 0.1,train_size= 0.9, random_state=42)


vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
#linesOfTrainData_Transformed =  vectorizer.fit_transform(features_train)
linesOfTrainData_Transformed =  vectorizer.fit_transform(linesOfTrainData)

print (len(vectorizer.vocabulary_)) # after loading vocab from 1 source
print(vectorizer.vocabulary_['younger'])
linesOfTrainData_Transformed =  vectorizer.fit_transform(linesOfTestData)
print (len(vectorizer.vocabulary_)) # after loading vocab from 2nd source
print(vectorizer.vocabulary_['younger'])


#linesOfTestData_Transformed = vectorizer.transform(features_test)
linesOfTestData_Transformed = vectorizer.transform(linesOfTestData)

21456
21379
21174
21106


In [6]:
print(linesOfTrainData[:1]) 
print('-------')
print(linesOfTrainData_Transformed[:1])

['+1\tthi book lif saver.  help abl go back track trends, answ pedy questions, commun diff tim night newborn.  think on thing everyon requir leav hospital.  went pag newborn version, mov inf version, fin second inf book (third total) right baby turn 1.  see thing must hav baby [...]\n']
-------
  (0, 13436)	0.14380652995
  (0, 12286)	0.151669834287
  (0, 13235)	0.251096266512
  (0, 898)	0.149984374319
  (0, 19150)	0.26806979078
  (0, 2055)	0.0831350167096
  (0, 7322)	0.243571043016
  (0, 16701)	0.150721320458
  (0, 5627)	0.149762694765
  (0, 3668)	0.153496830197
  (0, 15946)	0.327152400334
  (0, 8717)	0.192517434962
  (0, 11882)	0.129196387806
  (0, 10758)	0.364629492777
  (0, 11154)	0.145251646234
  (0, 6429)	0.102601807415
  (0, 6026)	0.28552781051
  (0, 1797)	0.217778738408
  (0, 14669)	0.282995160754
  (0, 8695)	0.329633853779
  (0, 15532)	0.144578324191


In [7]:

print(linesOfTestData[:1])
print('-------')
print(linesOfTestData_Transformed[:1])

["perfect new parents. abl keep track baby's feeding, sleep diap chang schedule first two half month lif. mad lif easy doct would ask quest habit right there!\n"]
-------
  (0, 19150)	0.26806979078
  (0, 16701)	0.150721320458
  (0, 15946)	0.327152400334
  (0, 15532)	0.144578324191
  (0, 14669)	0.282995160754
  (0, 13436)	0.14380652995
  (0, 13235)	0.251096266512
  (0, 12286)	0.151669834287
  (0, 11882)	0.129196387806
  (0, 11154)	0.145251646234
  (0, 10758)	0.364629492777
  (0, 8717)	0.192517434962
  (0, 8695)	0.329633853779
  (0, 7322)	0.243571043016
  (0, 6429)	0.102601807415
  (0, 6026)	0.28552781051
  (0, 5627)	0.149762694765
  (0, 3668)	0.153496830197
  (0, 2055)	0.0831350167096
  (0, 1797)	0.217778738408
  (0, 898)	0.149984374319


In [70]:


# selector = SelectPercentile(f_classif, percentile=10)
# selector.fit(linesOfTrainData_Transformed, labels_train) #labels_train
# linesOfTrainData_Transformed = selector.transform(linesOfTrainData_Transformed)
# linesOfTestData_Transformed = selector.transform(linesOfTestData_Transformed)

In [8]:
def CalculateCosine(vt,vs):
        dotProduct = vt.dot(np.transpose(vS))
        lengtht = np.linalg.norm(vt.data)
        lengthS = np.linalg.norm(vS.data)

        #handle exceptions

        if lengthS!=0 and lengtht!=0 :
            cosineSimilarityValue= dotProduct/(lengtht*lengthS)
        else:
            cosineSimilarityValue= 0
        return cosineSimilarityValue

In [ ]:


f = open('TestData/Test/format_out.dat', 'w')
for vt in linesOfTestData_Transformed:
    cosineSimilarityValues=[]
    for vS in linesOfTrainData_Transformed:
        cosineSimilarityValue = CalculateCosine(vt,vS)
        cosineSimilarityValues.append(cosineSimilarityValue)

    kneighbours = heapq.nlargest(3, cosineSimilarityValues)
    #kneighbours = sp.sparse.csr_matrix.max(cosineSimilarityValues)

    neighbourReviewTypeList = []
    neighbourReviewTypeNegative = 0
    neighbourReviewTypePositive = 0
    print("@@@@@")
    for neighbour in kneighbours:
        index = cosineSimilarityValues.index(neighbour.data[0])
        print("####",index)
        if linesOfTrainData[index].strip()[0] == '-':
            #neighbourReviewTypeList.append("-1")
            neighbourReviewTypeNegative+=1
        elif linesOfTrainData[index].strip()[0] == '+':
            #neighbourReviewTypeList.append("+1")
            neighbourReviewTypePositive+=1


    if neighbourReviewTypeNegative > neighbourReviewTypePositive:
        f.write('-1\n')
    else:
        f.write('+1\n')

print("-----")


/usr/local/lib/python3.5/dist-packages/scipy/sparse/compressed.py:225: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


@@@@@
#### 0
#### 3
#### 9
